# Modelo para Capstone

### Manejo de datos 

In [9]:
import pandas as pd
from gurobipy import *
import numpy as np


In [10]:
bdd_ventas = pd.read_excel("Base de Datos Bodega.xlsx")
bdd_proyeccion = pd.read_excel("Base de Datos Bodega.xlsx", sheet_name=1)
bdd_bodegas = pd.read_excel("Base de Datos Bodega.xlsx", sheet_name=2)
bdd_comunas = pd.read_excel("Base de Datos Bodega.xlsx", sheet_name=3)

In [11]:
# Pasamos la columna de fechas a formato de fechas
bdd_ventas['Fecha'] = pd.to_datetime(bdd_ventas['Fecha'])
bdd_proyeccion['Fecha'] =pd.to_datetime(bdd_proyeccion['Fecha'])

In [12]:
#Creamos el df de ventas agrupadas por cliente
bdd_ventas_agrupadas = bdd_ventas.groupby("ID Cliente")["Cantidad"].sum().reset_index()

In [16]:
bdd_ventas_agrupadas

,ID Cliente,Cantidad
0,1,1728.164816
1,2,878.660744
2,3,12311.242240
3,4,70756.899376
4,5,418.653512
...,...,...
2872,3123,93.600000
2873,3124,4.000000
2874,3125,16.000000
2875,3126,64.000000


In [18]:
# Revisamos si hay datos faltantes
#bdd_comunas.isna().any(axis=1).sum() 

In [55]:
bdd_demandas = bdd_ventas["ID Cliente"].value_counts() #Demandas por clientes

Armamos diccionarios con valores 

In [68]:
dict_bodegas = bdd_bodegas.to_dict(orient='index') #Keys del 0 al 9
dict_comunas = bdd_comunas.to_dict(orient='index') #keys del 0 al 342

In [67]:
dict_bodegas

{0: {'ID Bodega': 1, 'LAT': -29.96233333333333, 'LONG': -71.293},
 1: {'ID Bodega': 2, 'LAT': -38.50433333333334, 'LONG': -72.40633333333334},
 2: {'ID Bodega': 3, 'LAT': -32.885, 'LONG': -71.236},
 3: {'ID Bodega': 4, 'LAT': -36.92266666666666, 'LONG': -72.349},
 4: {'ID Bodega': 5, 'LAT': -35.85, 'LONG': -71.58166666666666},
 5: {'ID Bodega': 6, 'LAT': -34.95733333333333, 'LONG': -71.20933333333333},
 6: {'ID Bodega': 7, 'LAT': -34.58266666666667, 'LONG': -70.97966666666667},
 7: {'ID Bodega': 8, 'LAT': -34.138, 'LONG': -70.72566666666667},
 8: {'ID Bodega': 9, 'LAT': -40.55933333333333, 'LONG': -73.10733333333333},
 9: {'ID Bodega': 10, 'LAT': -36.599, 'LONG': -72.091}}

### Matrices de distancias

Acá deje los índices de las bodegas dentro del diccionario para evitar confusiones, porque parten en 1 pero las comunas en 0.

#### Distancias Manhattan

In [82]:
# Supongamos que tienes los diccionarios 'dict_bodegas' y 'dict_comunas'

# Radio aproximado de la Tierra en metros
radio_tierra = 6371000  # metros

# Crear una matriz de distancias con ceros
num_bodegas = len(dict_bodegas)
num_comunas = len(dict_comunas)
matriz_manhattan = np.zeros((num_bodegas, num_comunas))

# Llenar la matriz de distancias
for i, (bodega, bodega_coords) in enumerate(dict_bodegas.items()): #Revisamos todas las tuplas (key, valores) del diccionario de bodegas
    for j, (comuna, comuna_coords) in enumerate(dict_comunas.items()): #Revisamos todas las tuplas (key, valores) del diccionario de comunas

        # Sacamos la diferencia y hacemos la conversión a metros
        lat_diff = abs(bodega_coords['LAT'] - comuna_coords['LAT']) * (np.pi / 180) * radio_tierra
        lon_diff = abs(bodega_coords['LONG'] - comuna_coords['LON']) * (np.pi / 180) * radio_tierra * np.cos((bodega_coords['LAT'] + comuna_coords['LAT']) * 0.5 * (np.pi / 180)) 
        
        matriz_manhattan[i, j] = round((lat_diff + lon_diff)/1000, 2)  # Distancia Manhattan en metros, redondeado al segundo decimal

#### Distancias sobre la red

In [84]:
#Recomiendo descargar ocupando este código. Lo que viene a continuación demora harto en correr.

### Modelo